# Timeline

** Author : Sangkeun Jung (hugmanskj@gmail.com)**

Tensorflow support a tool for analyzing computation costs and resources name as **timeline**.
**timeline** is a 3rd party package. 

(For now - 2016-10-27) It only supports a first cycle of graph running.

## import timeline

```python
from tensorflow.python.client import timeline
```

## Prepare collecting meta data

After session creation, 

```python
    with tf.Session() as sess:
        # .. 
        sess.run(tf.initialize_all_variables())
        # ..
        
        run_metadata = tf.RunMetadata()  # <-- prepare collecting metadata
        trace_file = open('timeline.ctf.json', 'w') # <-- file to store

        for <training loop>:
            # ...
            _ = sess.run([train_op],
                           options=tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE),
                           run_metadata=run_metadata,
                       )
            trace = timeline.Timeline(step_stats=run_metadata.step_stats)
            trace_file.write(trace.generate_chrome_trace_format())
            
        trace_file.close()
```


## Visualizing traced file with Chrome

timline dump traced information as chrome trace file format. 
So you need to install chrome if it is not yet installed. 

- launch chrome browser
- Put below URL in the navigation
    - chrome://tracing
- The tracing screen looks like

<img src="./dnn_codes/fig/timeline_01.jpg">

- Then load traced file ( in this case, "timeline.ctf.json" )
- The visualized screen looks like 

<img src="./dnn_codes/fig/timeline_02.jpg">




## Problem Shooting!

Sometimes, timeline generates following errors 
```bash
I tensorflow/stream_executor/dso_loader.cc:102] Couldn't open CUDA library libcupti.so. LD_LIBRARY_PATH: /usr/local/lib:/usr/local/cuda/lib64:/usr/local/cuda-7.0/extras/CUPTI/lib64
F tensorflow/core/platform/default/gpu/cupti_wrapper.cc:58] Check failed: f != nullptr could not find cuptiActivityRegisterCallbacksin libcupti DSO; dlerror: /usr/local/lib/python2.7/dist-packages/tensorflow/python/_pywrap_tensorflow.so: undefined symbol: cuptiActivityRegisterCallbacks
```

Most of the case, your export enviroments do not connect correct cuda library. 
Correct as follows:

- check current environment 
    - <code>export | egrep 'LD_LIBRARY_PATH'</code>
    - in my case, it shows
    - <code> declare -x LD_LIBRARY_PATH="/usr/local/lib:/usr/local/cuda/lib64:/usr/local/cuda-7.5/extras/CUPTI/lib64" </code>
  
in this setting, tensorflow tries to use <code>/usr/local/cuda-7.5/extras/CUPTI/lib64"</code>. Check there is a <code>libcupti.so</code> in the folder. 